### feature합치기-- -1일과 -2일 비교 내용 삽입
#### investors, historical(매일 거래량 등) and 분류항목 합치기

In [1]:
import datetime, time
from datetime import date

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [2]:
# index column : date
def find_previous_date(df_inv, df_his, date_current): # 현재일에서 바로 직전 1~3거래일을 찾음.
    i = 0
    date2 = 0
    while True:
        i += 1
        date2 = date_current - datetime.timedelta(days = i)
        
#         try:
#             (df_inv.loc[date2]['retail'] != 0) & (df_his.loc[date2]['vol'] != '') # 2022년 01. 01.년 이전 날짜로 들어오면 return
#             return 0
#         except :
        if (df_inv.loc[date2]['retail'] != 0) & (df_his.loc[date2]['vol'] != '') :  
            # investors와 historical 두 개 데이터에 같이 없어야 함
            break
        if i >= 30:  # 현재일 대비 최대 30일전까지 거래가 있을 경우 계산 (현실적으로 최대 10일 이상 폐장되는 경우 없음)
            break

    return date2

In [3]:
# 거래일과 거래일 바로 이전일의 변화율을 계산함.
def find_ratio(df_o, date_current):
    df_o_trans = df_o.transpose()
    df_o_trans.columns = ['before', 'after']
    df_o_trans[date_current] = (df_o_trans['after'] - df_o_trans['before']) / df_o_trans['before']
    return df_o_trans.transpose()

In [4]:
# 변화율(historical, investors), weekday를 계산하고 합하여 return함
def combine_data(df_o, df_inv, df_his):
    # df_inv : df_investor, df_his: df_historical
    investor_rate = pd.DataFrame()
    historical_rate = pd.DataFrame()
    date_weekday = pd.DataFrame()
    current_previous_date = {} # 현재날짜와 하루 이전 날짜 보관
    
    # 최초의 0이 아닌(최초 거래가 이루어진 일자) 날짜로 부터 일주일 후부터 기준.
    # 최초 2일전까지의 날짜를 확보하기 위함. 현재일의 전날과 전전날의 이틀간 데이터 변화율 계산
    for k, i in enumerate(df_inv.iloc[:, 1]):  
        if i != 0: 
            skip_num = k + 10 # 8: 전일, 9: 전전일, 10: -3일까지 확보 
            break

    for date_current in df_o.index[skip_num:]: 
        # 최초 2일전까지의 날짜를 확보하기 위함. 현재일의 전날과 전전날의 이틀간 데이터 변화율 계산
        # 거래 전일(-1일) 날짜구하기

        # 거래가 없는날 (개인의 거래가 없는 날) 은 skip
        if df_investors_temp.set_index('date').loc[date_current].iloc[0] == 0: 
            continue
            
        date_previous_c = find_previous_date(df_inv, df_his, date_current)
#         if date_previous_c == 0 : continue # 2022년 1월 1일 이전날짜는 skip하기 위함
        current_previous_date[date_current] = date_previous_c

        # 거래 전날 요일 구하기
        date_temp = {'date': date_current, 'weekday' : date_previous_c.weekday()}
        df_temp = pd.DataFrame(date_temp, index=[0]).set_index('date')
        date_weekday = pd.concat([date_weekday, df_temp], axis=0)
        date_previous_1 = find_previous_date(df_inv, df_his, date_previous_c)
        # 거래 전날, 전전날을 확인하고 변화정도 계산하기(find_ratio)
        df_inv_comp_1 = df_inv.loc[[date_previous_1, date_previous_c]]
        df_his_comp_1 = df_his.loc[[date_previous_1, date_previous_c]]
    
    
        # 거래 전전일(-2일) 날짜 구하기
        date_previous_2 = find_previous_date(df_inv, df_his, date_previous_1)
        # 거래 전전날(-2일째) 요일 구하기  -- 2일전 요일은 의미 없을 것 같아서 추가하지 않음.
#         date_temp_2 = {'date': date_current, 'weekday' : date_previous_2.weekday()}
#         df_temp_2 = pd.DataFrame(date_temp_2, index=[0]).set_index('date')
#         date_weekday = pd.concat([date_weekday, df_temp_2], axis=0)
        # 거래 전날(-1일), 전전날(-3일)을 확인하고 변화정도 계산하기(find_ratio)
        df_inv_comp_2 = df_inv.loc[[date_previous_2, date_previous_c]]
        df_his_comp_2 = df_his.loc[[date_previous_2, date_previous_c]]
            
        # 전날 -전전날 ratio, 전날 - 전전전날 ratio, column확대
        df_inv_concat_2 = pd.concat([find_ratio(df_inv_comp_1, date_current).iloc[[-1]],
                                 find_ratio(df_inv_comp_2, date_current).iloc[[-1]]], axis=1)
        df_his_concat_2 = pd.concat([find_ratio(df_his_comp_1, date_current).iloc[[-1]],
                                 find_ratio(df_his_comp_2, date_current).iloc[[-1]]], axis=1)
        
        investor_rate = pd.concat([investor_rate,  df_inv_concat_2], axis=0)
        historical_rate = pd.concat([historical_rate, df_his_concat_2], axis=0)

#     total = pd.concat([investor_rate, historical_rate, date_weekday, df_o ], axis=1)
    total = pd.concat([investor_rate, historical_rate, date_weekday], axis=1)

    return total, current_previous_date

In [5]:
code = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'],
                 '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code = {'005930': ['삼성전자', 'sec']}

code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng'], '030200' : ['KT', 'kt']}
# code = {'005380' : ['현대차', 'hyunmotor']}

In [6]:
# hist_column = [ 'date', 'open', 'high', 'low', 'close', 'close_cr', 'vol']
hist_column_m = [ 'date', 'open', 'high', 'low', 'close', 'vol'] # close_cr 제외하고 사용. divided by zero 회피용
inv_column = ['date', 'retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc']

In [7]:
# investor.pkl, historical.pkl. 읽기

# ****************************************************************************************************
## 주요: 오늘날짜로 데이터를 분석하고자 하면 pkl directory 내의 .pkl 데이터가 오늘 날짜로 모두 update시켜 놓아야 함.
##   C:\jupyter연습\factors_affecting_stock_price' 디렉토리에 있는
#    'get_company_daily_price_since_last_update_to_pickle.ipynb' 과 (historical.pkl과 investors.pkl update 프로그램)
#     'get_common_data_since_last_update_to_pickle (미국, 한국 주요 지표 등 update용)
# ****************************************************************************************************
directory_for_ml_ex = './data/data_for_ml/expand_date/'
directory_for_ml_ex_update = './data/data_for_ml/expand_date/update/'
pkl_directory = './data/company_pkl/'

for key, val in code.items():
#     pkl_name= '{}{}_4class.pkl'.format(directory_for_ml, val[1])
#     df_class_temp = pd.read_pickle(pkl_name)
#     df_class_temp['date'] = pd.to_datetime(df_class_temp['date']) #change to timestamp
#     df_class_temp['date'] = df_class_temp['date'].dt.date  # change to datetime
 
    pkl_name= '{}_historical.pkl'.format(val[1])
    df_historical_temp = pd.read_pickle(pkl_directory + pkl_name)
    df_historical_temp = df_historical_temp[hist_column_m]

    # close_cr 행을 없앰(변동이 없는 경우가 빌생하여 divided zero error 발생), close_cr은 target column에서 재 계산하여사용
    df_historical_temp['date'] = df_historical_temp['date'].dt.date # change to datetime
    
    pkl_name= '{}_investors.pkl'.format(val[1])
    df_investors_temp = pd.read_pickle(pkl_directory + pkl_name)
    df_investors_temp['date'] = df_investors_temp['date'].dt.date # change to datetime
    
    # ******** 시작 일자, 마지막 일자  지정 ***********
    start_date = datetime.date(2022, 1, 1) # 2022년 01월 01일 자료 있음. 추후 이전날짜 추가시 수정 필요
    end_date = df_investors_temp['date'].iloc[-1]  # 투자자별 자료가 있는 마지막 날짜
    
    date_range_ts = pd.date_range(start=start_date, end=end_date)
    df_base = pd.DataFrame(pd.Series(date_range_ts, name='date'))
    df_base['date'] = df_base['date'].dt.date
    
# get combined data (eg. df_sec_combined, df_ktng_combined...)
#     df_combined_temp = combine_data(df_class_temp.set_index('date'), df_investors_temp.set_index('date'),
#                                     df_historical_temp.set_index('date'))

    df_combined_temp, cp_date = combine_data(df_base.set_index('date'), 
                                    df_investors_temp.set_index('date'), df_historical_temp.set_index('date'))

    # column nama change according to the newly added columns
    column_name_change = ['retail_1', 'foreigner_1', 'institution_1', 'financial_1', 'invtrust_1', 'pension_1', 
          'privequity_1', 'bank_1', 'insurance_1', 'financeetc_1', 'corporateetc_1', 'foreigneretc_1', 
          'retail_2', 'foreigner_2', 'institution_2', 'financial_2', 'invtrust_2', 'pension_2', 
          'privequity_2', 'bank_2', 'insurance_2', 'financeetc_2', 'corporateetc_2', 'foreigneretc_2', 
          'open_1', 'high_1', 'low_1', 'close_1', 'vol_1', 
          'open_2', 'high_2', 'low_2', 'close_2', 'vol_2', 'weekday' ]
    
    df_combined_temp.columns = column_name_change
    
    df_combined_temp['temp'] = df_combined_temp['close_1'].shift(-1) # 현재날짜 증감을 확인하기 위하여 임시 컬럼 추가
    # 마지막 row의 temp는 None이기 때문에 drop
    df_combined_temp = df_combined_temp.iloc[:-1]
    
    min_rate = 0.0 # +로 끝난 상황을 알기 위함
    df_combined_temp['cr_00'] = df_combined_temp['temp'].map(lambda x : 1 if x > min_rate else 0)
    min_rate = 0.005 # 수수료등 비용 0.2672% 이상 확인하기 위함, 0.5% 상승 마감
    df_combined_temp['cr_05'] = df_combined_temp['temp'].map(lambda x : 1 if x >= min_rate else 0)
    min_rate = 0.010 # 1.0% 상승 마감
    df_combined_temp['cr_10'] = df_combined_temp['temp'].map(lambda x : 1 if x >= min_rate else 0)
    min_rate = 0.015 # 1.5% 상승 마감
    df_combined_temp['cr_15'] = df_combined_temp['temp'].map(lambda x : 1 if x >= min_rate else 0)
    min_rate = 0.020 # 2.0% 상승 마감
    df_combined_temp['cr_20'] = df_combined_temp['temp'].map(lambda x : 1 if x >= min_rate else 0)
    
    df_combined_temp.drop(columns='temp', inplace=True) # 사용후 삭제
    
    column_selected = column_name_change
    column_selected.extend(['cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20'])  # 아래 class column 이 변경에 따라 수정해야 함
    
    globals()['df_{}_combined'.format(val[1])] = df_combined_temp.copy()
    globals()['df_{}_sel'.format(val[1])] = df_combined_temp[column_selected]

In [8]:
df_combined_temp.head()

,retail_1,foreigner_1,institution_1,financial_1,invtrust_1,pension_1,privequity_1,bank_1,insurance_1,financeetc_1,...,high_2,low_2,close_2,vol_2,weekday,cr_00,cr_05,cr_10,cr_15,cr_20
2022-01-13,3.299333,-0.551504,-1.780230,-5.881497,-2.165517,-0.999689,0.222222,inf,-15.320755,-1.598684,...,0.008091,0.013180,0.004870,0.164967,2,1,1,1,0,0
2022-01-14,1.564523,3.799366,0.452590,-0.806644,-4.695266,-1775.000000,4.415584,16.00000,-0.206851,-0.829670,...,0.012862,0.011438,0.021207,0.165628,3,0,0,0,0,0
2022-01-17,-0.516860,-0.534873,-0.617115,-6.396476,-0.612490,0.481962,-2.350120,-1.00000,-0.591362,1.451613,...,0.011236,0.013008,0.008078,-0.093971,4,0,0,0,0,0
2022-01-18,-0.772190,-0.581599,312.138968,-0.685306,-1.072314,-0.522632,-2.326821,inf,-1.369919,-1.000000,...,-0.001587,0.000000,-0.009585,13.330804,0,1,1,1,1,1
2022-01-19,5.769835,2.057094,-0.992483,-1.238651,-72.771429,-0.968924,0.385542,-1.00013,-19.043956,NaN,...,0.011111,0.001605,0.016026,0.296486,1,1,1,0,0,0


In [9]:
df_combined_temp.iloc[:, 25:].tail()

,high_1,low_1,close_1,vol_1,open_2,high_2,low_2,close_2,vol_2,weekday,cr_00,cr_05,cr_10,cr_15,cr_20
2023-03-20,0.003344,-0.008475,-0.001684,0.518006,0.013582,-0.003322,-0.006791,-0.014950,0.001338,4,1,1,1,1,1
2023-03-21,0.015000,0.018803,0.020236,0.022988,-0.003344,0.018395,0.010169,0.018519,0.552901,0,1,0,0,0,0
2023-03-22,0.003284,0.013423,0.004959,-0.159258,0.020101,0.018333,0.032479,0.025295,-0.139931,1,1,0,0,0,0
2023-03-23,0.000000,0.003311,0.001645,-0.266307,0.025168,0.003284,0.016779,0.006612,-0.383153,2,0,0,0,0,0
2023-03-24,-0.008183,-0.019802,-0.013136,1.202107,-0.004926,-0.008183,-0.016556,-0.011513,0.615671,3,0,0,0,0,0


In [10]:
'''
# 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.
column_selected = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc',
              'open', 'high', 'low', 'close', 'vol','weekday']
'''

"\n# 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.\ncolumn_selected = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',\n              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc',\n              'open', 'high', 'low', 'close', 'vol','weekday']\n"

## 주요 지표 읽기

### Read and Combine data 

In [ ]:
# common_pkl = ["bok_rate.pkl", "cpi.pkl", "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
#               "fear_greed.pkl", "fed_rate.pkl", "gold.pkl", "ixic_future.pkl", "kor_10yr_bond.pkl",
#               "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
#               "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
#               "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl"]

In [11]:
# 매일 변화가 있는 항목 추출; 금리변동과 같이 주기적을 발생하는 항목들(cpi, bok_rate, fed_rate 등)은 제외
common_pkl = [ "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
               "ixic_future.pkl", "kor_10yr_bond.pkl",
              "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
              "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
              "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl",
              'spsy.pkl', 'spny.pkl', 'spxhc.pkl', 'splrcd.pkl', 'splrci.pkl', 'splrcu.pkl', 'splrcs.pkl', 
              'splrct.pkl', 'splrcl.pkl', 'splrcm.pkl', 'ixbk.pkl', 'ixfn.pkl', 'ixid.pkl', 'ixis.pkl', 
              'ixk.pkl', 'ixtr.pkl', 'ixut.pkl', 'nbi.pkl', 'bkx.pkl' 
             ]

In [12]:
# common_sector_summary = ['spsy.pkl', 'spny.pkl', 'spxhc.pkl', 'splrcd.pkl', 'splrci.pkl', 'splrcu.pkl', 'splrcs.pkl', 
#                   'splrct.pkl', 'splrcl.pkl', 'splrcm.pkl', 'ixbk.pkl', 'ixfn.pkl', 'ixid.pkl', 'ixis.pkl', 
#                   'ixk.pkl', 'ixtr.pkl', 'ixut.pkl', 'nbi.pkl', 'bkx.pkl']

In [13]:
# dict로 만들어 pkl내에 있는 df의 column 이름과 매칭시키기 위하여 구성
common_col_name = {'dji.pkl':'dji', 'dji_future.pkl':'dji_f', 'dxy_future.pkl':'dxy', 
                   'ixic_future.pkl':'ixic_f', 'kor_10yr_bond.pkl':'bond_kor_10',
                   'kor_2yr_bond.pkl':'bond_kor_2', 'kosdaq.pkl':'kosdaq', 'kospi.pkl':'kospi',
                   'krw_rate.pkl':'krw', 'nas.pkl':'ixic', 'snp_future.pkl':'spx_f',
                   'sox.pkl':'sox', 'spx.pkl':'spx', 'us_10yr_bond.pkl':'bond_usa_10',
                   'us_2yr_bond.pkl':'bond_usa_2', 'us_3mon_bond.pkl':'bond_usa_3m',
                   'vix.pkl':'vix', 'wti_future.pkl':'wti',
                   'spsy.pkl':'spsy', 'spny.pkl':'spny', 'spxhc.pkl':'spxhc', 'splrcd.pkl':'splrcd', 
                   'splrci.pkl':'splrci', 'splrcu.pkl':'splrcu', 'splrcs.pkl':'splrcs', 
                   'splrct.pkl':'splrct', 'splrcl.pkl':'splrcl', 'splrcm.pkl':'splrcm', 
                   'ixbk.pkl':'ixbk', 'ixfn.pkl':'ixfn', 'ixid.pkl':'ixid', 'ixis.pkl':'ixis', 
                   'ixk.pkl':'ixk', 'ixtr.pkl':'ixtr', 'ixut.pkl':'ixut', 'nbi.pkl':'nbi', 'bkx.pkl':'bkx'
                   }

In [14]:
end_date = datetime.datetime.today().date()
start_date = '20220101'  # 2022년 01월 01일 자료 있음. 추후 이전날짜 추가시 수정 필요
date_range_ts = pd.date_range(start=start_date, end=end_date)

In [15]:
# pkl_common_directory = './data/common_pkl/'
# df_base = pd.DataFrame(pd.Series(date_range_ts, name='date')).set_index('date')
# # for index_name in major_index:
# for index_name in common_pkl:
#     key_name = index_name[:-4]
# #     globals()['df_{}'.format(key_name)] = pd.read_pickle(pkl_common_directory + index_name)
# #     df_base = df_base.merge(globals()['df_{}'.format(key_name)].set_index('date').iloc[:, [-1]], \
# #                             how='left', left_index=True, right_index=True)
#     df_temp = pd.read_pickle(pkl_common_directory + index_name)
#     df_temp['date'] = df_temp['date'] + datetime.timedelta(days = 1) # 하루전 데이터와 일치시키기 위하여 하루 shift
#     df_base = df_base.merge(df_temp.set_index('date').iloc[:, [-1]], \
#                             how='left', left_index=True, right_index=True)

In [16]:
pkl_common_directory = './data/common_pkl/'
df_base = pd.DataFrame(pd.Series(date_range_ts, name='date')).set_index('date')
# df_base_t = pd.DataFrame(pd.Series(date_range_ts, name='date'))['date'].dt.date
# df_base = df_base_t.set_index('date', inplace=True)

date_compare = pd.DataFrame.from_dict(cp_date, orient='index')
date_compare.reset_index(inplace=True)
date_compare.columns = ['date_c', 'date_p'] #현재날짜와 하루 전 날짜
# date_compare.astype('datetime64[ns]')

for index_name in common_pkl:
    key_name = index_name[:-4]
    col_name = common_col_name[index_name]
    df_temp = pd.read_pickle(pkl_common_directory + index_name)
    df_temp['date'] = df_temp['date'].dt.date # date type으로 변환
    df_temp['temp'] = df_temp[col_name].shift(2)  # 2일전 데이터와 비교하기 위하여 shift
    df_temp[f'{col_name}_cr_2'] = (df_temp[col_name] - df_temp['temp'])/df_temp['temp']*100 # 하루전 데이터는 "_cr"
#     df_temp['date'] = df_temp['date'] + datetime.timedelta(days = 1) # 하루전 데이터와 일치시키기 위하여 하루 shift
    
    df_new = df_temp[df_temp['date'].isin(date_compare['date_p'])] # 전날 비교날짜가 있는 row만 선택
    # 하루전 날짜와 매칭한 df 생성 
    add_c = list(date_compare[date_compare['date_p'].isin(df_new['date'])]['date_c'])
    df_new.insert(len(df_new.columns)-1, 'date_c', value=add_c)
    df_new.set_index('date_c', inplace=True)
    
    df_base = df_base.merge(df_new.iloc[:, [-3, -1]], \
                            how='left', left_index=True, right_index=True)

In [17]:
df_new

,date,bkx,open,high,low,volume,bkx_cr,temp,bkx_cr_2
date_c,,,,,,,,,
2022-01-13,2022-01-12,147.279999,147.039993,148.100006,146.529999,0,0.50%,145.250000,1.397590
2022-01-14,2022-01-13,147.559998,147.839996,148.960007,147.139999,0,0.19%,146.539993,0.696059
2022-01-17,2022-01-14,147.100006,145.869995,147.160004,144.119995,0,-0.31%,147.279999,-0.122211
2022-01-19,2022-01-18,143.570007,147.139999,147.479996,142.940002,0,-2.40%,147.559998,-2.703978
2022-01-20,2022-01-19,139.070007,144.600006,144.690002,139.039993,0,-3.13%,147.100006,-5.458870
...,...,...,...,...,...,...,...,...,...
2023-03-21,2023-03-20,79.419998,79.690002,81.900002,79.180000,0,0.79%,83.169998,-4.508837
2023-03-22,2023-03-21,83.349998,81.550003,84.000000,81.550003,0,4.95%,78.800003,5.774106
2023-03-23,2023-03-22,79.430000,83.239998,83.250000,79.400002,0,-4.70%,79.419998,0.012594


In [ ]:
'''
df_sec_sel = df_sec_combined[column_selected]
'''

In [18]:
for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])] \
    = globals()['df_{}_sel'.format(val[1])].merge(df_base, how='left', left_index=True, right_index=True)

In [22]:
ll = df_kt_sel.columns

Index(['retail_1', 'foreigner_1', 'institution_1', 'financial_1', 'invtrust_1',
       'pension_1', 'privequity_1', 'bank_1', 'insurance_1', 'financeetc_1',
       ...
       'ixk_cr', 'ixk_cr_2', 'ixtr_cr', 'ixtr_cr_2', 'ixut_cr', 'ixut_cr_2',
       'nbi_cr', 'nbi_cr_2', 'bkx_cr', 'bkx_cr_2'],
      dtype='object', length=114)

In [50]:
col_inv1 = ['retail_1', 'foreigner_1', 'institution_1', 'financial_1', 'invtrust_1', 'pension_1', 'privequity_1', 
            'bank_1', 'insurance_1', 'financeetc_1', 'corporateetc_1', 'foreigneretc_1']

In [51]:
col_inv2 = ['retail_2', 'foreigner_2', 'institution_2', 'financial_2', 'invtrust_2', 'pension_2',
            'privequity_2', 'bank_2', 'insurance_2', 'financeetc_2', 'corporateetc_2', 'foreigneretc_2']

In [52]:
col_his1 = ['open_1', 'high_1', 'low_1', 'close_1', 'vol_1']

In [53]:
col_his2 = ['open_2', 'high_2', 'low_2', 'close_2', 'vol_2']

In [54]:
col_cr = ['weekday', 'cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20']

In [55]:
col_common1 = ["dji_cr", "dji_f_cr", "dxy_cr", "ixic_f_cr", "bond_kor_10_cr", "bond_kor_2_cr", "kosdaq_cr", "kospi_cr", 
         "krw_cr", "ixic_cr", "spx_f_cr", "sox_cr", "spx_cr", "bond_usa_10_cr", "bond_usa_2_cr", "bond_usa_3m_cr", 
         "vix_cr", "wti_cr", "spsy_cr", "spny_cr", "spxhc_cr", "splrcd_cr", "splrci_cr", "splrcu_cr", "splrcs_cr",
         "splrct_cr", "splrcl_cr", "splrcm_cr", "ixbk_cr", "ixfn_cr", "ixid_cr", "ixis_cr", "ixk_cr", "ixtr_cr",
         "ixut_cr", "nbi_cr", "bkx_cr"]

In [56]:
col_common2 = ["dji_cr_2", "dji_f_cr_2", "dxy_cr_2", "ixic_f_cr_2", "bond_kor_10_cr_2", "bond_kor_2_cr_2", "kosdaq_cr_2", "kospi_cr_2",
         "krw_cr_2", "ixic_cr_2", "spx_f_cr_2", "sox_cr_2", "spx_cr_2", "bond_usa_10_cr_2", "bond_usa_2_cr_2", "bond_usa_3m_cr_2",
         "vix_cr_2", "wti_cr_2", "spsy_cr_2", "spny_cr_2", "spxhc_cr_2", "splrcd_cr_2", "splrci_cr_2", "splrcu_cr_2",
         "splrcs_cr_2", "splrct_cr_2", "splrcl_cr_2", "splrcm_cr_2", "ixbk_cr_2", "ixfn_cr_2", "ixid_cr_2",
         "ixis_cr_2", "ixk_cr_2", "ixtr_cr_2", "ixut_cr_2", "nbi_cr_2", "bkx_cr_2"]

In [57]:
col_col = col_inv1 + col_common1 + col_his1 + col_inv2 + col_common2 + col_his2 + col_cr

In [58]:
'''
df_sec_sel = df_sec_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_hyunmotor_sel = df_hyunmotor_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_naver_sel = df_naver_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_ktng_sel = df_ktng_sel.merge(df_base, how='left', left_index=True, right_index=True)
'''

"\ndf_sec_sel = df_sec_sel.merge(df_base, how='left', left_index=True, right_index=True)\ndf_hyunmotor_sel = df_hyunmotor_sel.merge(df_base, how='left', left_index=True, right_index=True)\ndf_naver_sel = df_naver_sel.merge(df_base, how='left', left_index=True, right_index=True)\ndf_ktng_sel = df_ktng_sel.merge(df_base, how='left', left_index=True, right_index=True)\n"

In [59]:
# # 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.
# # weekday, 'cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20' column을 마지막으로 이동 
# new_columns = ['retail_1', 'foreigner_1', 'institution_1', 'financial_1', 'invtrust_1', 'pension_1', 
#                'privequity_1', 'bank_1', 'insurance_1', 'financeetc_1', 'corporateetc_1', 'foreigneretc_1', 
#                'dji_cr', 'dji_f_cr', 'ixic_cr', 'ixic_f_cr', 'spx_cr', 'spx_f_cr', 'bond_kor_10_cr',
#                'bond_kor_2_cr', 'dxy_cr', 'bond_usa_10_cr','bond_usa_2_cr', 'bond_usa_3m_cr',
#                'kosdaq_cr', 'kospi_cr', 'krw_cr', 'sox_cr', 'vix_cr', 'wti_cr',                
#                'open_1', 'high_1', 'low_1', 'close_1', 'vol_1',      
#                'retail_2', 'foreigner_2', 'institution_2', 'financial_2', 'invtrust_2', 'pension_2',
#                'privequity_2', 'bank_2', 'insurance_2', 'financeetc_2', 'corporateetc_2', 'foreigneretc_2',
#                'dji_cr_2', 'dji_f_cr_2', 'ixic_cr_2', 'ixic_f_cr_2', 'spx_cr_2', 'spx_f_cr_2',
#                'bond_kor_10_cr_2', 'bond_kor_2_cr_2', 'dxy_cr_2', 'bond_usa_10_cr_2','bond_usa_2_cr_2', 
#                'bond_usa_3m_cr_2', 'kosdaq_cr_2', 'kospi_cr_2', 'krw_cr_2', 'sox_cr_2', 'vix_cr_2', 'wti_cr_2', 
# #                 'open', 'high', 'low', 'close', 'vol', 'weekday',  # close column 제거함 (class column 계산항목으로 사용됨)
#                 'open_2', 'high_2', 'low_2', 'close_2', 'vol_2', 'weekday', 
#                 'cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20']

In [60]:
new_columns = col_col

In [68]:
# column 순서 변경 : weekday, 'cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20' column을 마지막으로 이동 
for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])] = globals()['df_{}_sel'.format(val[1])][new_columns]

In [69]:
df_hyunmotor_sel.iloc[:, 10:20].head(20)

,corporateetc_1,foreigneretc_1,dji_cr,dji_f_cr,dxy_cr,ixic_f_cr,bond_kor_10_cr,bond_kor_2_cr,kosdaq_cr,kospi_cr


In [63]:
'''
# drop inf, -inf : replace inf to 1 or -1 . 데이터를 살리고
# 변동률이 무한대가 되는 것을 방지하기 위해서, 나중에 발생하는 에러를 방지
df_sec_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)

.impute 사용하는 것을 고려할 필요 있음.
'''

'\n# drop inf, -inf : replace inf to 1 or -1 . 데이터를 살리고\n# 변동률이 무한대가 되는 것을 방지하기 위해서, 나중에 발생하는 에러를 방지\ndf_sec_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)\n\n.impute 사용하는 것을 고려할 필요 있음.\n'

In [64]:
# drop inf, -inf : replace inf to 1 or -1 . 데이터를 살리고
# 변동률이 무한대가 되는 것을 방지하기 위해서, 나중에 발생하는 에러를 방지
for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])].replace([np.inf, -np.inf], [1, -1], inplace=True)

# **** 데이터 전처리 from sklearn.impute import SimpleImputer, SimpleImputer 사용하기로 함.

In [65]:
# # delete rows which include NaN : dji, spx, nasdaq 지수가 nan인 rows 제거
# df_sec_sel.dropna(inplace=True)

In [66]:
# delete rows which include NaN : dji, spx, nasdaq 지수가 nan인 rows 제거
for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])].dropna(inplace=True)

In [67]:
df_hyunmotor_sel.head()

,retail_1,foreigner_1,institution_1,financial_1,invtrust_1,pension_1,privequity_1,bank_1,insurance_1,financeetc_1,...,high_2,low_2,close_2,vol_2,weekday,cr_00,cr_05,cr_10,cr_15,cr_20


In [ ]:
# string column을 numeric으로 전환
def string_to_num(df):
    df.replace('%', '', regex=True, inplace=True)
    return df.apply(pd.to_numeric)
'''
df_sec_sel = string_to_num(df_sec_sel)
'''

for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])] = string_to_num(globals()['df_{}_sel'.format(val[1])])

In [ ]:
# pickle로 데이터 저장
'''
directory_for_ml_ex_update = './data/data_for_ml/expand_date/update/'
df_sec_sel.to_pickle(directory_for_ml_ex_update + 'df_sec_sel.pkl')
df_sec_sel.to_csv(directory_for_ml_ex_update + 'df_sec_sel.csv')
'''

for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])].to_pickle(directory_for_ml_ex + 'df_{}_sel.pkl'.format(val[1]))
    globals()['df_{}_sel'.format(val[1])].to_csv(directory_for_ml_ex + 'df_{}_sel.csv'.format(val[1]))